In [1]:
import pandas as pd

# Loading the dataset and Checking the information about the fields and attributes present in them.

In [23]:
flight= pd.read_excel("D:\\Make_my_trip\\FinalDataset\\FlightFINALdataset.xlsx")

In [24]:
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40552 entries, 0 to 40551
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   travelCode       40552 non-null  int64  
 1   User_ID          40552 non-null  int64  
 2   Departure        40552 non-null  object 
 3   Arrival          40552 non-null  object 
 4   flightType       40552 non-null  object 
 5   Flight_price     40552 non-null  float64
 6   Flight_duration  40552 non-null  float64
 7   Flight_Distance  40552 non-null  float64
 8   Flight_agency    40552 non-null  object 
 9   Departure_date   40552 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 3.1+ MB


In [26]:
flight.to_csv("flight.csv",index=False)

Importing travelCode into Trips Table

In [ ]:
import mysql.connector
import pandas as pd
import os
# Load the CSV file into a DataFrame
df = pd.read_csv("flight.csv")

# Ensure travelCode and User_ID are converted to Python int

df["travelCode"] = df["travelCode"].astype(int).map(int)  # Convert NumPy int64 to Python int
df["User_ID"] = df["User_ID"].astype(int).map(int)  # Convert NumPy int64 to Python int


# Establish MySQL database connection
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)

cursor = conn.cursor()

### Insert Unique travelCode into Trips Table ###
# Get unique travelCode and user_id pairs
unique_trips = df[['travelCode', 'User_ID']].drop_duplicates()

# SQL query to insert unique travelCode into Trips
insert_trips_query = """
INSERT IGNORE INTO Trips (travelCode, user_id) VALUES (%s, %s)
"""
batch_size = 500  # Insert 500 rows at a time
batch = []

for i, row in unique_trips.iterrows():
    batch.append((int(row["travelCode"]), int(row["User_ID"])))

    if len(batch) >= batch_size:
        cursor.executemany(insert_trips_query, batch)
        conn.commit()  # Commit every 500 records
        batch = []  # Reset batch

# Insert any remaining records
if batch:
    cursor.executemany(insert_trips_query, batch)
    conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Travel codes successfully imported into Trips table!")

Travel codes successfully imported into Trips table!


In [25]:
value_counts = flight['flightType'].value_counts()
print(value_counts)

flightType
firstClass    17304
premium       11656
economic      11592
Name: count, dtype: int64


In [ ]:
import mysql.connector
import pandas as pd
import os 

# Load CSV into DataFrame
df = pd.read_csv("flight.csv")

# Convert to standard Python types to avoid MySQL errors
df["travelCode"] = df["travelCode"].astype(int)
df["User_ID"] = df["User_ID"].astype(int)
df["flightType"] = df["flightType"].astype(str)
df["Flight_price"] = df["Flight_price"].astype(float)
df["Flight_duration"] = df["Flight_duration"].astype(float)
df["Flight_Distance"] = df["Flight_Distance"].astype(float)
df["Departure_date"] = pd.to_datetime(df["Departure_date"], errors='coerce')

# Drop rows with NaN values in essential columns to prevent errors
df = df.dropna(subset=["travelCode", "User_ID", "Departure_date"])

# Convert dataframe rows into a list of tuples (ensuring no NumPy types)
data = [
    (
        int(row.travelCode), 
        int(row.User_ID), 
        row.Departure, 
        row.Arrival, 
        row.flightType, 
        float(row.Flight_price), 
        float(row.Flight_duration), 
        float(row.Flight_Distance), 
        row.Flight_agency, 
        row.Departure_date.strftime('%Y-%m-%d %H:%M:%S')  # Ensure correct MySQL datetime format
    ) 
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert SQL Query
insert_query = """
INSERT INTO Flight (
    travelCode, User_ID, Departure, Arrival, flightType, Flight_price, 
    Flight_duration, Flight_Distance, Flight_agency, Departure_date
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch insert to avoid timeout issues
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()
    print(f"Inserted {i + len(batch)} records into Flight table...")

# Close connections
cursor.close()
conn.close()

print("All flight data inserted successfully!")

C:\Users\hetvi\AppData\Local\Temp\ipykernel_16672\635643734.py:14: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Departure_date"] = pd.to_datetime(df["Departure_date"], errors='coerce')


Inserted 1000 records into Flight table...
Inserted 2000 records into Flight table...
Inserted 3000 records into Flight table...
Inserted 4000 records into Flight table...
Inserted 5000 records into Flight table...
Inserted 6000 records into Flight table...
Inserted 7000 records into Flight table...
Inserted 8000 records into Flight table...
Inserted 9000 records into Flight table...
Inserted 10000 records into Flight table...
Inserted 11000 records into Flight table...
Inserted 12000 records into Flight table...
Inserted 13000 records into Flight table...
Inserted 14000 records into Flight table...
Inserted 15000 records into Flight table...
Inserted 16000 records into Flight table...
Inserted 17000 records into Flight table...
Inserted 18000 records into Flight table...
Inserted 19000 records into Flight table...
Inserted 20000 records into Flight table...
Inserted 21000 records into Flight table...
Inserted 22000 records into Flight table...
Inserted 23000 records into Flight table.

In [47]:
hotel=pd.read_excel(r"D:\Make_my_trip\FinalDataset\HotelFINALdataset.xlsx")

In [6]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40552 entries, 0 to 40551
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   User_ID              40552 non-null  int64  
 1   travelCode           40552 non-null  int64  
 2   Hotel_Name           40552 non-null  object 
 3   Arrival_place        40552 non-null  object 
 4   Hotel_stay           40552 non-null  int64  
 5   Hotel_per_day_price  40552 non-null  float64
 6   Check-in             40552 non-null  object 
 7   Hotel_TotalPrice     40552 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 2.5+ MB


In [11]:
hotel.head()

,User_ID,travelCode,Hotel_Name,Arrival_place,Hotel_stay,Hotel_per_day_price,Check-in,Hotel_TotalPrice
0,0,0,Hotel A,Florianopolis (SC),4,313.02,09/26/2019,1252.08
1,0,2,Hotel K,Salvador (BH),2,263.41,10/10/2019,526.82
2,0,7,Hotel K,Salvador (BH),3,263.41,11/14/2019,790.23
3,0,11,Hotel K,Salvador (BH),4,263.41,12/12/2019,1053.64
4,0,13,Hotel A,Florianopolis (SC),1,313.02,12/26/2019,313.02


In [ ]:
import mysql.connector
import pandas as pd
import os

# Establish connection to MySQL database
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)

# Load Trips table into a DataFrame
query = "SELECT travelCode, user_id FROM Trips"
df_trips = pd.read_sql(query, conn)

# Load hotel.csv into a DataFrame
df_hotel = pd.read_excel(r"D:\Make_my_trip\FinalDataset\HotelFINALdataset.xlsx")

# Ensure column names match
df_hotel.rename(columns={'Check-in': 'Check_in'}, inplace=True)

# Convert travelCode to integer (remove any spaces)
df_hotel['travelCode'] = df_hotel['travelCode'].astype(str).str.strip().astype(int)

# Merge data to check for mismatches
df_merged = df_hotel[['travelCode', 'User_ID']].merge(df_trips, 
                                                       left_on=['travelCode', 'User_ID'], 
                                                       right_on=['travelCode', 'user_id'], 
                                                       how='left', indicator=True)

# Find mismatched rows (where _merge column is "left_only", meaning they are missing from Trips)
df_mismatched = df_merged[df_merged['_merge'] == 'left_only'][['travelCode', 'User_ID']]

# Display results
if not df_mismatched.empty:
    print("❌ The following travelCode and User_ID pairs from hotel.csv are missing in Trips:")
    display(df_mismatched)
else:
    print("✅ All travelCode and User_ID pairs in hotel.csv exist in Trips.")

# Close the connection
conn.close()


C:\Users\hetvi\AppData\Local\Temp\ipykernel_16672\1657733891.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_trips = pd.read_sql(query, conn)


✅ All travelCode and User_ID pairs in hotel.csv exist in Trips.


In [48]:
df = pd.read_excel(r"D:\Make_my_trip\FinalDataset\Hotel_1Final.xlsx")
df.to_csv("hotel.csv",index=False)

In [49]:
unique_hotels = df["Hotel_Name"].unique()
print(f"Number of unique hotels: {len(unique_hotels)}")
print(unique_hotels)


Number of unique hotels: 27
['The Sapphire Retreat' 'Oceanview Inn' 'Horizon Cliff Hotel'
 'Emerald Bay Resort' 'Regal Heights Hotel' 'Urban Elegance Hotel'
 'Harbor View Hotel' 'Seaside Retreat' 'Royal Orchid' 'Grand Palace Hotel'
 'Summit Heights Hotel' 'Cityscape Hotel' 'Sunset Resort'
 'Golden Sands Hotel' 'Starlight Inn' 'Lakeside Manor' 'Mountain Lodge'
 'Evergreen Lodge' 'Velvet Sunset Inn' 'Crystal Cove Resort'
 'Serenity Bay' 'The Majestic Tower' 'Aurora Bliss Hotel'
 'Azure Waves Resort' 'Skyline Suites' 'Tranquil Meadows Hotel'
 'The Grand Horizon']


In [50]:
duplicate_hotels = df.groupby("Hotel_Name")["Arrival_place"].nunique()
conflicting_hotels = duplicate_hotels[duplicate_hotels > 1]

if not conflicting_hotels.empty:
    print("The following hotels appear in multiple cities:")
    print(conflicting_hotels)
else:
    print("No hotel is listed under multiple arrival places. Your recommendation logic is safe!")


No hotel is listed under multiple arrival places. Your recommendation logic is safe!


In [51]:
unique_arrival_places = df["Arrival_place"].unique()
print(f"Number of unique arrival places: {len(unique_arrival_places)}")
print(unique_arrival_places)


Number of unique arrival places: 9
['Florianopolis (SC)' 'Salvador (BH)' 'Natal (RN)' 'Aracaju (SE)'
 'Recife (PE)' 'Sao Paulo (SP)' 'Campo Grande (MS)' 'Rio de Janeiro (RJ)'
 'Brasilia (DF)']


In [52]:
city_hotel_mapping = df.groupby("Hotel_Name")["Arrival_place"].nunique()
conflicting_hotels = city_hotel_mapping[city_hotel_mapping > 1]

if not conflicting_hotels.empty:
    print("⚠️ The following hotels appear in multiple cities:")
    print(conflicting_hotels)
else:
    print("✅ Each hotel name is assigned to only one city. Your recommendation logic is safe!")


✅ Each hotel name is assigned to only one city. Your recommendation logic is safe!


In [53]:
hotel_city_distribution = df.groupby(["Arrival_place", "Hotel_Name"]).size().reset_index(name="Count")
print(hotel_city_distribution)


          Arrival_place              Hotel_Name  Count
0          Aracaju (SE)      Azure Waves Resort   1400
1          Aracaju (SE)     Regal Heights Hotel   1406
2          Aracaju (SE)       Velvet Sunset Inn   1399
3         Brasilia (DF)           Starlight Inn   1444
4         Brasilia (DF)       The Grand Horizon   1461
5         Brasilia (DF)  Tranquil Meadows Hotel   1532
6     Campo Grande (MS)         Evergreen Lodge   1455
7     Campo Grande (MS)            Serenity Bay   1429
8     Campo Grande (MS)    Summit Heights Hotel   1449
9    Florianopolis (SC)      Aurora Bliss Hotel   1090
10   Florianopolis (SC)     Horizon Cliff Hotel   1132
11   Florianopolis (SC)    The Sapphire Retreat   1108
12           Natal (RN)         Cityscape Hotel   1613
13           Natal (RN)      Emerald Bay Resort   1616
14           Natal (RN)            Royal Orchid   1600
15          Recife (PE)     Crystal Cove Resort   1510
16          Recife (PE)          Lakeside Manor   1495
17        

In [ ]:
import mysql.connector
import pandas as pd
import os

# Load CSV into DataFrame
df = pd.read_csv("hotel.csv")

# Convert to standard Python types to avoid MySQL errors
df["travelCode"] = df["travelCode"].astype(int)
df["User_ID"] = df["User_ID"].astype(int)
df["Hotel_Name"] = df["Hotel_Name"].astype(str)
df["Arrival_place"] = df["Arrival_place"].astype(str)
df["Hotel_stay"] = df["Hotel_stay"].astype(int)
df["Hotel_per_day_price"] = df["Hotel_per_day_price"].astype(float)
df.rename(columns={"Check-in": "Check_in"}, inplace=True)
df["Check_in"] = pd.to_datetime(df["Check_in"], errors='coerce')
df["Hotel_TotalPrice"] = df["Hotel_TotalPrice"].astype(float)

# Drop rows with NaN values in essential columns
df = df.dropna(subset=["travelCode", "User_ID", "Check_in"])

# Convert dataframe rows into a list of tuples
data = [
    (
        int(row.travelCode), 
        int(row.User_ID), 
        row.Hotel_Name, 
        row.Arrival_place, 
        int(row.Hotel_stay), 
        float(row.Hotel_per_day_price), 
        row.Check_in.strftime('%Y-%m-%d %H:%M:%S'),  # Ensure correct MySQL datetime format
        float(row.Hotel_TotalPrice)
    ) 
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
   host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert SQL Query
insert_query = """
INSERT INTO Hotel (
    travelCode, User_ID, Hotel_Name, Arrival_place, Hotel_stay, 
    Hotel_per_day_price, Check_in, Hotel_TotalPrice
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch insert to avoid timeout issues
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()
    print(f"Inserted {i + len(batch)} records into Hotel table...")

# Close connections
cursor.close()
conn.close()

print("All hotel data inserted successfully!")


Inserted 1000 records into Hotel table...
Inserted 2000 records into Hotel table...
Inserted 3000 records into Hotel table...
Inserted 4000 records into Hotel table...
Inserted 5000 records into Hotel table...
Inserted 6000 records into Hotel table...
Inserted 7000 records into Hotel table...
Inserted 8000 records into Hotel table...
Inserted 9000 records into Hotel table...
Inserted 10000 records into Hotel table...
Inserted 11000 records into Hotel table...
Inserted 12000 records into Hotel table...
Inserted 13000 records into Hotel table...
Inserted 14000 records into Hotel table...
Inserted 15000 records into Hotel table...
Inserted 16000 records into Hotel table...
Inserted 17000 records into Hotel table...
Inserted 18000 records into Hotel table...
Inserted 19000 records into Hotel table...
Inserted 20000 records into Hotel table...
Inserted 21000 records into Hotel table...
Inserted 22000 records into Hotel table...
Inserted 23000 records into Hotel table...
Inserted 24000 recor

In [3]:
import pandas as pd

In [ ]:
car=pd.read_excel(r"D:\Make_my_trip\FinalDataset\CarFINALdataset.xlsx")

In [37]:
car.to_csv("car.csv",index=False)

In [5]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   User_ID             3773 non-null   int64 
 1   travelCode          3773 non-null   int64 
 2   Check-in            3773 non-null   object
 3   pickupLocation      3773 non-null   object
 4   dropoffLocation     3773 non-null   object
 5   carType             3773 non-null   object
 6   rentalAgency        3773 non-null   object
 7   rentalDuration      3773 non-null   int64 
 8   Car_total_distance  3773 non-null   int64 
 9   fuelPolicy          3773 non-null   object
 10  Car_bookingStatus   3773 non-null   object
 11  total_rent_price    3773 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 353.8+ KB


In [7]:
car_value_counts = car['pickupLocation'].value_counts()
print(car_value_counts)

pickupLocation
East Michael     9
Lake John        7
New Michael      6
South William    5
East John        5
                ..
New Jonstad      1
Mccannton        1
Lake Erika       1
Jonesshire       1
Kylehaven        1
Name: count, Length: 3347, dtype: int64


In [ ]:
import mysql.connector
import pandas as pd
import os

# Load CSV into DataFrame
df = pd.read_csv("car.csv")

# Rename columns to match the database schema
df.rename(columns={"Check-in": "Check_in"}, inplace=True)

# Convert data types to match MySQL table structure
df["User_ID"] = df["User_ID"].astype(int)
df["travelCode"] = df["travelCode"].astype(int)
df["Check_in"] = pd.to_datetime(df["Check_in"], errors='coerce')
df["rentalDuration"] = df["rentalDuration"].astype(int)
df["Car_total_distance"] = df["Car_total_distance"].astype(int)
df["total_rent_price"] = df["total_rent_price"].astype(int)

# Drop rows with NaN values in essential columns
df = df.dropna(subset=["travelCode", "User_ID", "Check_in"])

# Convert dataframe rows into a list of tuples (avoiding NumPy types)
data = [
    (
        int(row.User_ID),
        int(row.travelCode),
        row.Check_in.strftime('%Y-%m-%d %H:%M:%S'),  # Ensure correct MySQL datetime format
        row.pickupLocation,
        row.dropoffLocation,
        row.carType,
        row.rentalAgency,
        int(row.rentalDuration),
        int(row.Car_total_distance),
        row.fuelPolicy,
        row.Car_bookingStatus,
        int(row.total_rent_price)
    )
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert SQL Query
insert_query = """
INSERT INTO CarRental (
    User_ID, travelCode, Check_in, pickupLocation, dropoffLocation, carType, 
    rentalAgency, rentalDuration, Car_total_distance, fuelPolicy, 
    Car_bookingStatus, total_rent_price
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch insert to avoid timeout issues
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()
    print(f"Inserted {i + len(batch)} records into CarRental table...")

# Close connections
cursor.close()
conn.close()

print("All car rental data inserted successfully!")


Inserted 1000 records into CarRental table...
Inserted 2000 records into CarRental table...
Inserted 3000 records into CarRental table...
Inserted 3773 records into CarRental table...
All car rental data inserted successfully!


In [40]:
passenger=pd.read_excel(r"D:\Make_my_trip\FinalDataset\PassengerFINALdataset.xlsx")
passenger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User_ID   1310 non-null   int64 
 1   company   1310 non-null   object
 2   Name      1310 non-null   object
 3   gender_x  1310 non-null   object
dtypes: int64(1), object(3)
memory usage: 41.1+ KB


In [ ]:
import mysql.connector
import pandas as pd
import os

# Load CSV into DataFrame
df = pd.read_excel(r"D:\Make_my_trip\FinalDataset\PassengerFINALdataset.xlsx")

# Convert data types to match MySQL table structure
df["User_ID"] = df["User_ID"].astype(int)
df["company"] = df["company"].astype(str)
df["Name"] = df["Name"].astype(str)
df["gender_x"] = df["gender_x"].astype(str)

# Drop rows with NaN values in essential columns
df = df.dropna(subset=["User_ID", "Name"])

# Convert dataframe rows into a list of tuples (avoiding NumPy types)
data = [
    (
        int(row.User_ID),
        row.company,
        row.Name,
        row.gender_x
    )
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert SQL Query
insert_query = """
INSERT INTO Passenger (User_ID, company, Name, gender_x) 
VALUES (%s, %s, %s, %s)
"""

# Batch insert to avoid timeout issues
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()
    print(f"Inserted {i + len(batch)} records into Passenger table...")

# Close connections
cursor.close()
conn.close()

print("All passenger data inserted successfully!")


Inserted 1000 records into Passenger table...
Inserted 1310 records into Passenger table...
All passenger data inserted successfully!


In [42]:
cust_call=pd.read_excel(r"D:\Make_my_trip\FinalDataset\CustomerCallFINALdataset.xlsx")
cust_call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6219 entries, 0 to 6218
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   User_ID           6219 non-null   int64 
 1   Arrival_date      6219 non-null   object
 2   issueType         6219 non-null   object
 3   resolutionStatus  6219 non-null   object
 4   supervisorID      6219 non-null   int64 
 5   Call_Date         6219 non-null   object
dtypes: int64(2), object(4)
memory usage: 291.6+ KB


In [ ]:
import mysql.connector
import pandas as pd
import os

# Load CSV into DataFrame
df = pd.read_excel(r"D:\Make_my_trip\FinalDataset\CustomerCallFINALdataset.xlsx")

# Convert data types
df["User_ID"] = df["User_ID"].astype(int)
df["supervisorID"] = df["supervisorID"].astype(int)
df["Arrival_date"] = pd.to_datetime(df["Arrival_date"], errors='coerce')
df["Call_Date"] = pd.to_datetime(df["Call_Date"], errors='coerce')

# Remove duplicates to avoid primary key conflicts
df = df.drop_duplicates(subset=["User_ID", "Call_Date"])

# Convert dataframe to list of tuples
data = [
    (
        int(row.User_ID), 
        row.Arrival_date.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(row.Arrival_date) else None,
        row.issueType, 
        row.resolutionStatus, 
        int(row.supervisorID), 
        row.Call_Date.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(row.Call_Date) else None
    ) 
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert Query with IGNORE
insert_query = """
INSERT IGNORE INTO CustomerCall (User_ID, Arrival_date, issueType, resolutionStatus, supervisorID, Call_Date) 
VALUES (%s, %s, %s, %s, %s, %s)
"""

# Reduce batch size and commit frequently
batch_size = 500  
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()
    print(f"Inserted {i + len(batch)} records into CustomerCall table...")

# Close connections
cursor.close()
conn.close()

print("All customer call data inserted successfully!")

Inserted 500 records into CustomerCall table...
Inserted 1000 records into CustomerCall table...
Inserted 1500 records into CustomerCall table...
Inserted 2000 records into CustomerCall table...
Inserted 2369 records into CustomerCall table...
All customer call data inserted successfully!


In [55]:
guest=pd.read_excel(r"D:\Make_my_trip\FinalDataset\GuestFINALdataset.xlsx")
guest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10175 entries, 0 to 10174
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Guest_ID       10175 non-null  int64 
 1   travelCode     10175 non-null  int64 
 2   Guest_name     10175 non-null  object
 3   Guest_Gender   10175 non-null  object
 4   Age            10175 non-null  int64 
 5   Guest_PhoneNo  10175 non-null  object
 6   Guest_email    10175 non-null  object
 7   idProof        10175 non-null  object
dtypes: int64(3), object(5)
memory usage: 636.1+ KB


In [ ]:
import mysql.connector
import pandas as pd
import os

# Load CSV into DataFrame
df = pd.read_excel(r"D:\Make_my_trip\FinalDataset\GuestFINALdataset.xlsx")

# Convert data types to match MySQL table
df["Guest_ID"] = df["Guest_ID"].astype(int)
df["travelCode"] = df["travelCode"].astype(int)
df["Guest_name"] = df["Guest_name"].astype(str)
df["Guest_Gender"] = df["Guest_Gender"].astype(str)
df["Age"] = df["Age"].astype(int)
df["Guest_PhoneNo"] = df["Guest_PhoneNo"].astype(str)
df["Guest_email"] = df["Guest_email"].astype(str)
df["idProof"] = df["idProof"].astype(str)

# Drop duplicates based on Guest_ID (avoid primary key conflicts)
df = df.drop_duplicates(subset=["Guest_ID"])

# Convert dataframe to list of tuples
data = [
    (
        int(row.Guest_ID), 
        int(row.travelCode), 
        row.Guest_name, 
        row.Guest_Gender, 
        int(row.Age), 
        row.Guest_PhoneNo, 
        row.Guest_email, 
        row.idProof
    ) 
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert Query (Ignore Duplicates)
insert_query = """
INSERT IGNORE INTO Guest (Guest_ID, travelCode, Guest_name, Guest_Gender, Age, Guest_PhoneNo, Guest_email, idProof) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch Insert (Prevent Locking Issues)
batch_size = 100  
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()  # Commit after each batch
    print(f"Inserted {i + len(batch)} records into Guest table...")

# Close connections
cursor.close()
conn.close()

print("All guest data inserted successfully!")


Inserted 100 records into Guest table...
Inserted 200 records into Guest table...
Inserted 300 records into Guest table...
Inserted 400 records into Guest table...
Inserted 500 records into Guest table...
Inserted 600 records into Guest table...
Inserted 700 records into Guest table...
Inserted 800 records into Guest table...
Inserted 900 records into Guest table...
Inserted 1000 records into Guest table...
Inserted 1100 records into Guest table...
Inserted 1200 records into Guest table...
Inserted 1300 records into Guest table...
Inserted 1339 records into Guest table...
All guest data inserted successfully!


In [57]:
review=pd.read_excel(r"D:\Make_my_trip\FinalDataset\ReviewFINALdataset.xlsx")
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40552 entries, 0 to 40551
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   travelCode      40552 non-null  int64  
 1   User_ID         40552 non-null  int64  
 2   Car_rented      40552 non-null  object 
 3   review_car      3773 non-null   object 
 4   review_hotel    40552 non-null  object 
 5   review_flights  40552 non-null  object 
 6   flight_rating   40552 non-null  int64  
 7   hotel_rating    40552 non-null  int64  
 8   car_rating      40552 non-null  int64  
 9   overall_rating  40552 non-null  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 3.1+ MB


In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import os

# Load CSV into DataFrame
df =pd.read_excel(r"D:\Make_my_trip\FinalDataset\ReviewFINALdataset.xlsx")

# Convert data types to match MySQL table
df["travelCode"] = df["travelCode"].astype(int)
df["User_ID"] = df["User_ID"].astype(int)
df["Car_rented"] = df["Car_rented"].astype(str)
df["review_car"] = df["review_car"].fillna("").astype(str)  # Handle NULL values
df["review_hotel"] = df["review_hotel"].fillna("").astype(str)
df["review_flights"] = df["review_flights"].fillna("").astype(str)
df["flight_rating"] = df["flight_rating"].astype(int)
df["hotel_rating"] = df["hotel_rating"].astype(int)
df["car_rating"] = df["car_rating"].astype(int)
df["overall_rating"] = df["overall_rating"].astype(float)

# Drop duplicates based on travelCode and User_ID (avoid primary key conflicts)
df = df.drop_duplicates(subset=["travelCode", "User_ID"])

# Convert dataframe to list of tuples
data = [
    (
        int(row.travelCode), 
        int(row.User_ID), 
        row.Car_rented if row.Car_rented in ['Confirmed', 'Not Rented', 'Pending', 'Cancelled'] else 'Not Rented', 
        row.review_car, 
        row.review_hotel, 
        row.review_flights, 
        int(row.flight_rating), 
        int(row.hotel_rating), 
        int(row.car_rating), 
        float(row.overall_rating)
    ) 
    for row in df.itertuples(index=False)
]

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"), 
    user=os.getenv("DB_USER"),
    password= os.getenv("DB_PASSWORD"),
    database="makemytrip"
)
cursor = conn.cursor()

# Insert Query (Ignore Duplicate Errors)
insert_query = """
INSERT IGNORE INTO Review (travelCode, User_ID, Car_rented, review_car, review_hotel, review_flights, 
                          flight_rating, hotel_rating, car_rating, overall_rating) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch Insert (Prevent Locking Issues)
batch_size = 1000  
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()  # Commit after each batch
    print(f"Inserted {i + len(batch)} records into Review table...")

# Close connections
cursor.close()
conn.close()

print("All review data inserted successfully!")


Inserted 1000 records into Review table...
Inserted 2000 records into Review table...
Inserted 3000 records into Review table...
Inserted 4000 records into Review table...
Inserted 5000 records into Review table...
Inserted 6000 records into Review table...
Inserted 7000 records into Review table...
Inserted 8000 records into Review table...
Inserted 9000 records into Review table...
Inserted 10000 records into Review table...
Inserted 11000 records into Review table...
Inserted 12000 records into Review table...
Inserted 13000 records into Review table...
Inserted 14000 records into Review table...
Inserted 15000 records into Review table...
Inserted 16000 records into Review table...
Inserted 17000 records into Review table...
Inserted 18000 records into Review table...
Inserted 19000 records into Review table...
Inserted 20000 records into Review table...
Inserted 21000 records into Review table...
Inserted 22000 records into Review table...
Inserted 23000 records into Review table.

In [ ]:
review_value_counts = review['Car_rented'].value_counts()
print(review_value_counts)

In [ ]:
gender_value_counts = user_1['gender'].value_counts()
print(gender_value_counts)

In [15]:
user=pd.read_csv(r"D:\Make_my_trip\FinalDataset\user_2.csv")
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               1340 non-null   int64  
 1   company               1340 non-null   object 
 2   name                  1340 non-null   object 
 3   gender                1340 non-null   object 
 4   age                   1340 non-null   int64  
 5   HotelBookingReviews   1340 non-null   object 
 6   FlightBookingReviews  1340 non-null   object 
 7   CarRentalReviews      1340 non-null   object 
 8   Rating                1340 non-null   int64  
 9   travelCode            1310 non-null   float64
dtypes: float64(1), int64(3), object(6)
memory usage: 104.8+ KB


In [16]:
user.columns

Index(['user_id', 'company', 'name', 'gender', 'age', 'HotelBookingReviews',
       'FlightBookingReviews', 'CarRentalReviews', 'Rating', 'travelCode'],
      dtype='object')

In [17]:
user.shape

(1340, 10)

In [18]:
user=user.drop(columns=['HotelBookingReviews',
       'FlightBookingReviews', 'CarRentalReviews', 'Rating', 'travelCode'],axis=True)

In [19]:
user.columns

Index(['user_id', 'company', 'name', 'gender', 'age'], dtype='object')

In [20]:
user.head()

,user_id,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


In [21]:
gender_value_counts = user['gender'].value_counts()
print(gender_value_counts)

gender
male      452
female    448
none      440
Name: count, dtype: int64


In [22]:
unique_values = user['gender'].unique()
unique_values

array(['male', 'female', 'none'], dtype=object)

In [11]:
# Clean and validate gender values
def clean_gender(value):
    value = str(value).strip()  # Remove spaces
    valid_genders = {"Male", "Female", "Other"}
    gender_map = {"M": "Male", "F": "Female", "O": "Other"}  # Mapping shorthand to full names
    return gender_map.get(value, value) if value in valid_genders or value in gender_map else None

user["gender"] = user["gender"].apply(clean_gender)

In [12]:
unique_values = user['gender'].unique()
unique_values

array([None], dtype=object)

In [13]:
user.head()

,user_id,company,name,gender,age
0,0,4You,Roy Braun,None,21
1,1,4You,Joseph Holsten,None,37
2,2,4You,Wilma Mcinnis,None,48
3,3,4You,Paula Daniel,None,23
4,4,4You,Patricia Carson,None,44
